# Lightweight Fine-Tuning Project

TODO: In this cell, describe your choices for each of the following

* PEFT technique: 
* Model: 
* Evaluation approach: 
* Fine-tuning dataset: 

## Loading and Evaluating a Foundation Model

TODO: In the cells below, load your chosen pre-trained Hugging Face model and evaluate its performance prior to fine-tuning. This step includes loading an appropriate tokenizer and dataset.

In [39]:
import torch 
import pandas as pd
import sklearn



from transformers import DistilBertTokenizer, DistilBertForSequenceClassification, BitsAndBytesConfig, Trainer, TrainingArguments, DataCollatorWithPadding
from datasets import load_dataset
from peft import LoraConfig, get_peft_model, TaskType
from sklearn.metrics import accuracy_score

In [20]:
#BitsAndBytesConfig??

In [10]:
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_compute_dtype=torch.float16,
    bnb_4bit_quant_type="nf4"
)

In [38]:
model = DistilBertForSequenceClassification.from_pretrained(
    "distilbert-base-uncased", num_labels=2,
    id2label={0:'NEGATIVE', 1:'POSITIVE'},
    label2id ={'NEGATIVE':0, 'POSITIVE':1},
    quantization_config=bnb_config
)
tokenizer = DistilBertTokenizer.from_pretrained("distilbert-base-uncased-finetuned-sst-2-english")

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['pre_classifier.bias', 'pre_classifier.weight', 'classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


### Specifically using the `Twitter Sentiment Analysis Training Corpus (TSATC)` for data from a large socia media platofrm

In [12]:
dataset = load_dataset("carblacac/twitter-sentiment-analysis")

/opt/conda/lib/python3.10/site-packages/datasets/load.py:1461: FutureWarning: The repository for carblacac/twitter-sentiment-analysis contains custom code which must be executed to correctly load the dataset. You can inspect the repository content at https://hf.co/datasets/carblacac/twitter-sentiment-analysis
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this dataset from the next major release of `datasets`.
  warnings.warn(


Generating train split: 0 examples [00:00, ? examples/s]

Generating test split: 0 examples [00:00, ? examples/s]

Map:   0%|          | 0/149985 [00:00<?, ? examples/s]

Map:   0%|          | 0/61998 [00:00<?, ? examples/s]

...

Creating json from Arrow format:   0%|          | 0/30 [00:00<?, ?ba/s]

Creating json from Arrow format:   0%|          | 0/62 [00:00<?, ?ba/s]

Generating train split:   0%|          | 0/119988 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/29997 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/61998 [00:00<?, ? examples/s]

In [33]:
eval_data = dataset['validation']
eval_data

Dataset({
    features: ['text', 'feeling'],
    num_rows: 29997
})

In [13]:
dataset

DatasetDict({
    train: Dataset({
        features: ['text', 'feeling'],
        num_rows: 119988
    })
    validation: Dataset({
        features: ['text', 'feeling'],
        num_rows: 29997
    })
    test: Dataset({
        features: ['text', 'feeling'],
        num_rows: 61998
    })
})

Printing the first `5` rows of texts and sentiments from our `training` subset

In [14]:
for x in range(0,6):
    print(f"{x}, {dataset['test'][x]}")
    print('--------------')

0, {'text': "@justineville ...yeahhh. ) i'm 39 tweets from 1,600!", 'feeling': 1}
--------------
1, {'text': '@ApplesnFeathers aww. Poor baby! On your only REAL day off.', 'feeling': 0}
--------------
2, {'text': '@joeymcintyre With my refunded $225 (Australian ticket price) I bought me a hot pair of brown boots  Woulda rathered seeing U any day', 'feeling': 0}
--------------
3, {'text': "It's fine. Today sucks just because me those things. i dunno if i can see you", 'feeling': 0}
--------------
4, {'text': 'Im just chilling on psp and stuff, but sitting on pc now, also watching wimledon, getting ready for holiday @WhiteTigerNora Ahh poor you', 'feeling': 0}
--------------
5, {'text': '@lisarinna very sad Lisa...she is freeeeeeeeeeee an Angel in Heaven xoxo', 'feeling': 0}
--------------


In [15]:
set(dataset['train']['feeling'])

{0, 1}

#### In summary, there are only 2 feelings really; '0 for Negative', '1 for Positive'

In [40]:
def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=1)
    return {'accuracy':(predictions == labels).mean()}

In [ ]:
training_args = TrainingArguments(


)

In [ ]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset["train"],
    eval_dataset=eval_data,
    tokenizer=tokenizer,
)

## Performing Parameter-Efficient Fine-Tuning

TODO: In the cells below, create a PEFT model from your loaded model, run a training loop, and save the PEFT model weights.

## Performing Inference with a PEFT Model

TODO: In the cells below, load the saved PEFT model weights and evaluate the performance of the trained PEFT model. Be sure to compare the results to the results from prior to fine-tuning.